Note: this notebook is set up to run with the env.yml containing the name 'polaris_datasets'

# Dataset creation with [Polaris](https://github.com/polaris-hub/polaris)
The first step of creating a benchmark is to set up a standard dataset which allows accessing the curated dataset (which has been demonstrated in <01_AZ_HLM_data_curation.ipynb>), and all necessary information about the dataset such as data source, description of endpoints, units etc. 

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import pandas as pd
import datamol as dm

# polaris dataset
from polaris.dataset import Dataset, ColumnAnnotation

from polaris.utils.types import HubOwner


root = pathlib.Path("__file__").absolute().parents[3]
os.chdir(root)
sys.path.insert(0, str(root))
from utils.docs_utils import load_readme

In [2]:
# Get the owner and organization
org = "polaris"
data_name = "human_HLM"
dirname = dm.fs.join(root, f"org-{org}", "astra_zeneca", data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/astra_zeneca/{data_name}"

owner = HubOwner(slug=org, type="organization")
owner

HubOwner(slug='polaris', external_id=None, type='organization')

In [3]:
BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_DIR = f"{gcp_root}/datasets"
FIGURE_DIR =  f"{gcp_root}/figures"

## Load existing data

In [4]:
PATH = f'{gcp_root}/data/curation/{data_name}_curated.csv'
table = pd.read_csv(PATH)
table.columns

Index(['SMILES', 'HLM_CLEARANCE', 'MOL_smiles', 'MOL_molhash_id',
       'MOL_molhash_id_no_stereo', 'MOL_num_stereoisomers',
       'MOL_num_undefined_stereoisomers', 'MOL_num_defined_stereo_center',
       'MOL_num_undefined_stereo_center', 'MOL_num_stereo_center',
       'MOL_undefined_E_D', 'MOL_undefined_E/Z', 'OUTLIER_HLM_CLEARANCE',
       'AC_HLM_CLEARANCE'],
      dtype='object')

### Below we specify the meta information of data columns

In [12]:
# Additional meta-data on the column level
annotations = {
    "MOL_smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    "HLM_CLEARANCE": ColumnAnnotation(
        description="Intrinsic clearance measured in human liver microsomes following incubation at 37C.",
        user_attributes={"unit": "uL/min/mg", "organism": "human"},
    ),
}

### Define `Dataset` object

In [6]:
version = "v1"
dataset_name = f"{data_name}-{version}"

In [15]:
dataset = Dataset(
    # The table is the core data-structure required to construct a dataset
    table=table[annotations.keys() ],
    # Additional meta-data on the dataset level.
    name=dataset_name,
    description="Intrinsic clearance measured in human liver microsomes experiment data released by AstraZeneca",
    source="https://www.ebi.ac.uk/chembl/document_report_card/CHEMBL3301370/",
    annotations=annotations,
    tags=["ADME"],
    owner=owner,
    license="CC-BY-SA-4.0",
    user_attributes={"year": "2016"},
    readme = load_readme("org-Polaris/astra_zeneca/human_HLM/AZ_human_HLM_readme.md"),
    curation_reference="https://github.com/polaris-hub/polaris-recipes/org-Polaris/astra_zeneca/human_HLM/01_AZ_human_HLM_data_curation.ipynb",
)

In [16]:
# save the dataset to GCP
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset_path = dataset.to_json(SAVE_DIR)
dataset_path

'gs://polaris-public/polaris-recipes/org-polaris/astra_zeneca/human_HLM/datasets/human_HLM-v1/dataset.json'

In [17]:
# upload to Polaris Hub
# dataset.upload_to_hub(owner=owner, access="private")